##### Training DeepCDR with the new data generator

So far we have written down a new generator function with tf.keras.Sequence, and we have trained the DeepCDR model with this function. We however still need to write an improve-compliant code if we are to share the work with people from ANL. Therefore, let's try doing that with the new generator function for both the train and inference scripts.

The generator function is available in the py script "New_data_generator_with_tf.py", and the train script with this function is available at "DeepCDR_train_with_new_generator.py". 

Okay, we have made the two scripts improve compliant: The codes are stored as; train script - "deepcdr_train_improve_with_new_generator.py" and infer script - "deepcdr_infer_improve_with_new_data_generator.py".

One more thing to do here before we get to the bootstrapping method and storing the predictions. See if the train and validation steps are necessary, or if the written generator will automatically complete training at the end of the required number of batches in each epoch

##### Training bootsrap samples for DeepCDR for UQ

Okay, we can now move to bootstrap sample training. We should adjust our code in a way that the trained model and the validation scores from the exercise can be stored, so that later we can use these models to compute the CIs and talk about uncertainity.

Let's use the bootstrap generator, and compare it against the original generator, to make sure the bootstrap sampling generator is in fact doing what it is supposed to do. This comparison is done in the 'Examine_bootstrap_samples.ipynb'.

We ensured that bootstrap samples indeed give different results than the regular training batches, so I think we are finally ready to implement the bootstrap training for DeepCDR for the UQ project - To get the predictions for the rrain data for the different botstrap samples, and then train the NNe, and then get the CIs, and thereby talk about the width and the coverages of the confidence intervals.

Notice that, we add the bootstrap data generator class to the python script "New_data_generator_with_tf.py" so that it would seem less complex to someone going through the codes (else too many python scripts, eventhough they are just simple fucntion r class definitions).

The bootstrap train code is in the phton script "DeepCDR_train_with_new_generator_bootstraps.py". This is a generic code, and is still not the improve-compliant one. This code defines the validation data generator (that is created by the regular generator construct and not the bootstrap), and the folder to store the bootstrap results outside the for loop for training the bootstrap models. Inside the for loop, it first defines the model, compile the model, train the model, save the model, evaluate the model, and also stores the validation predictions and the associated scores. However, we cannot store the predictions or the scores for the train data for the bootstrap work due to an assertion error that we get in the improve library (it basically checks if the train order of the predicted values is the same, but since in bootstraps we have some samples repeated, we will not be able to use the improve methods to store the predictions for the train data. Therefore, we will do this step in the next model train - NNe work).

Our next step will be to write improve-compliant code for training the bootrap models. 

The improve-compliant bootstrap training of the DeepCDR model is in the py script "deepcdr_train_bootstrap_improve_with_new_generator.py". Note that the build model path [Req] in the original code is now defined inside the for loop. We still need to make sure that this code works, and then we should be good to get started with developing the code for the model NNe.